In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

## Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [2]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [8]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {c:i for i, c in enumerate(world_set)}

In [9]:
print(vocab)

{'b': 0, 'a': 1, 'u': 2, 'e': 3, 'i': 4, '.': 5, 'p': 6, 'f': 7, 'm': 8, 'B': 9, 'h': 10, ',': 11, 't': 12, 'r': 13, 'g': 14, 'y': 15, 'k': 16, 'd': 17, ' ': 18, 'w': 19, 'l': 20, 'o': 21, 's': 22, "'": 23, 'c': 24, 'n': 25}


In [10]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [11]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [14]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i + sequence_length]
  y_str = sentence[i + 1: i + sequence_length + 1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Brick wall -> rick walls
1 rick walls -> ick walls 
2 ick walls  -> ck walls a
3 ck walls a -> k walls ar
4 k walls ar ->  walls are
5  walls are -> walls are 
6 walls are  -> alls are t
7 alls are t -> lls are th
8 lls are th -> ls are the
9 ls are the -> s are ther
10 s are ther ->  are there
11  are there -> are there 
12 are there  -> re there f
13 re there f -> e there fo
14 e there fo ->  there for
15  there for -> there for 
16 there for  -> here for a
17 here for a -> ere for a 
18 ere for a  -> re for a r
19 re for a r -> e for a re
20 e for a re ->  for a rea
21  for a rea -> for a reas
22 for a reas -> or a reaso
23 or a reaso -> r a reason
24 r a reason ->  a reason 
25  a reason  -> a reason a
26 a reason a ->  reason an
27  reason an -> reason and
28 reason and -> eason and 
29 eason and  -> ason and y
30 ason and y -> son and yo
31 son and yo -> on and you
32 on and you -> n and you 
33 n and you  ->  and you m
34  and you m -> and you mu
35 and you mu -> nd you mus
36

In [15]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[9, 13, 4, 24, 16, 18, 19, 1, 20, 20]
[13, 4, 24, 16, 18, 19, 1, 20, 20, 22]


In [18]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  if __name__ == '__main__':


In [19]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([188, 10, 26])
레이블의 크기 : torch.Size([188, 10])


In [20]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0.,

In [21]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([13,  4, 24, 16, 18, 19,  1, 20, 20, 22])


In [24]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)
  
  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x

In [25]:
net = Net(vocab_size, hidden_size, 2)

In [26]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [27]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([188, 10, 26])


In [30]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

kaaakaaaakkamaaaaakaaaaaaaaaakamamaabaaaaaaaamaabaaaaaabkaaamaaaakaabaakaaaakkamaamaaaaakaaaaaakkmmaakaaaakaabaaaaaakaaabaaaakaaaaaaamaaaakaabaakaaaakkamaaaaakaaaaaamamkaikaaakaabaakakkaaabaaaaakam
t    aaaa  a    aa   a   a a      a   a  aa   aa  aaaa   aaa aaaa  a    aaaa  a     araa   aa a    a  a  a aa aa  aa  a  aaa  a a aaa aaaa  a    aaaa  a    aa   aa a a aa  aa aaa  a aa   a a aa    
t                                                                                                                                                                                                    
t                                                                                                                                                                                                    
                                                                                                                                                                                                     
          

In [31]:
predict_str

"rick walls are there tor a reason and you must not think that the brick walls are 't there to keep us out, but rather in this way that the brick walls are there to khow us how badly we want thinks."

결과가 어떤가요?? 마지막 에폭의 문장이 그럴싸한가요?

## Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요! 

영어가 아닌 한국어로 시도해보는 것도 좋겠죠? 

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

## 첫 번째

In [32]:
import torch
import torch.nn as nn
import torch.optim as optim

In [86]:
sentence = ("And I can take you for a ride"
            "I had a premonition that we fell into a rhythm"
            "Where the music don't stop for life"
            )

In [87]:
char_set = list(set(sentence))  # 중복 제거한 문자 집합 생성
char_dic = {c: i for i, c in enumerate(char_set)} # 각 문자에 정수 인코딩

In [88]:
print(char_dic) # 공백도 여기서는 하나의 원소

{'W': 0, 'I': 1, 'a': 2, 'u': 3, 'e': 4, 'i': 5, 'p': 6, 'f': 7, 'm': 8, 'h': 9, 't': 10, 'r': 11, 'y': 12, 'k': 13, 'd': 14, ' ': 15, 'w': 16, 'l': 17, 'A': 18, 'o': 19, 's': 20, "'": 21, 'c': 22, 'n': 23}


In [89]:
dic_size = len(char_dic)
print('문자 집합의 크기 : {}'.format(dic_size))

문자 집합의 크기 : 24


In [90]:
# 하이퍼파라미터 설정
hidden_size = dic_size
sequence_length = 10
learning_rate = 0.1

In [91]:
# 데이터 구성
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i : (i+sequence_length)]
  y_str = sentence[i+1 : (i+sequence_length+1)]
  print(i, x_str, '->', y_str)
  
  x_data.append([char_dic[c] for c in x_str]) # x str to index
  y_data.append([char_dic[d] for d in y_str]) # y str to index

0 And I can  -> nd I can t
1 nd I can t -> d I can ta
2 d I can ta ->  I can tak
3  I can tak -> I can take
4 I can take ->  can take 
5  can take  -> can take y
6 can take y -> an take yo
7 an take yo -> n take you
8 n take you ->  take you 
9  take you  -> take you f
10 take you f -> ake you fo
11 ake you fo -> ke you for
12 ke you for -> e you for 
13 e you for  ->  you for a
14  you for a -> you for a 
15 you for a  -> ou for a r
16 ou for a r -> u for a ri
17 u for a ri ->  for a rid
18  for a rid -> for a ride
19 for a ride -> or a rideI
20 or a rideI -> r a rideI 
21 r a rideI  ->  a rideI h
22  a rideI h -> a rideI ha
23 a rideI ha ->  rideI had
24  rideI had -> rideI had 
25 rideI had  -> ideI had a
26 ideI had a -> deI had a 
27 deI had a  -> eI had a p
28 eI had a p -> I had a pr
29 I had a pr ->  had a pre
30  had a pre -> had a prem
31 had a prem -> ad a premo
32 ad a premo -> d a premon
33 d a premon ->  a premoni
34  a premoni -> a premonit
35 a premonit ->  premoniti
36

In [92]:
print(x_data[0])
print(y_data[0])

[18, 23, 14, 15, 1, 15, 22, 2, 23, 15]
[23, 14, 15, 1, 15, 22, 2, 23, 15, 10]


In [93]:
x_one_hot = [np.eye(dic_size)[x] for x in x_data] # x data를 one-hot encoding
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [94]:
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([100, 10, 24])
레이블의 크기 : torch.Size([100, 10])


In [95]:
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [96]:
print(Y[0])

tensor([23, 14, 15,  1, 15, 22,  2, 23, 15, 10])


In [97]:
class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):  # 현재 hidden_size는 dic_size와 같음
      super(Net, self).__init__()
      self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
      self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)
  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x

In [98]:
net = Net(dic_size, hidden_size, 2) # 두 개의 층

In [99]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [100]:
outputs = net(X)
print(outputs.shape)  # 3차원 텐서

torch.Size([100, 10, 24])


In [101]:
print(outputs.view(-1, dic_size).shape) # 2차원 텐서로 변환

torch.Size([1000, 24])


In [102]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([100, 10])
torch.Size([1000])


In [103]:
for i in range(100):
  optimizer.zero_grad()
  outputs = net(X)
  loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
  loss.backward()
  optimizer.step()

  results = outputs.argmax(dim=1)
  predict_str = ""
  for j, result in enumerate(results):
    if j == 0:
      predict_str += ''.join([char_set[t] for t in result])
    else:
      predict_str += char_set[result[-1]]
  
  print(predict_str)

uWWIfuWmaWephueiiuIeumeeuuaIufpieuuhmfpihmfpieupieuhmfpihmfpieuaeuaIaIpieuafhmfphmfpieuaIumfpihufpieuihmfpieuaIieuhmfpieuaI
eapaIWeiWWWuWfefaWWWeiWpieuhmfhhmfphmfpieuaimfhhmfphmhmhmfpiehmfpieuahmfphhmfpipieepihmfphmhhmfpiemfphhhmfpieupieuhmfhhmfpi
iihmfIpWppWWiWppIWiapiWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWWW
mmpIpummephhimeueImpemmhiehiehiehmmiieuhimmiimimieihiemimhmmeieuihmmuihemhhmfhiememimWemhimuheeiiemmfhimuiimuimWimhhmuiiemh
iiuiIfiuieuIuiifihifiiipppWIupppufpiIuuWWppWpWpppuuhIpupIWpupfIpWhufpWhuppWuufIupIupuufpWWppfpupupuupphuufIpupppWWpWpuuuuuW
ihhpIIhiimuWieiIhipIhehWeIWmWmiIWauWIWWWuWIWuWWWiWuIIWuWIWmfuifuWIWapWIWapiaWhmfaIiWfWIaWfWpiIuafpapImIWfaIWuWuWmWIfIfafWaW
uhemIhfhupIIueiIhupIieumIIfmfpiIufuWIuiWufIWmfmfmWppmfIfmfmfuifpiImfuiIuipufufmufpiiummfWmIuiIuffpifpiIuiIIfIWuhmfIipifuuiW
uueaIefhupmIueiIuIpaieimIphmfpieufIiIuiWufIWmfpfmuphmfpfpfmfhmfpihmfpieuiIufuihmfpimuWmfumIpieuifpmfpieuiIIWIWuhmfpiIuiuuiW
ufifWefh

## 두 번째
https://wikidocs.net/45101

In [104]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [124]:
text = ("어터슨 변호사는 결코 환하게 웃는 법이 없는 묵뚝뚝한 표정을 가진 사람이었다"
        "그는 야윈 데다가 키가 크고 무미건조하고 음울해 보이기까지 했다"
        "그럼에도 어터슨에게는 어딘가 매력적인 부분이 있었다")

In [125]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
vocab_size = len(tokenizer.word_index) + 1
print('단어 집합의 크기: {}'.format(vocab_size))

단어 집합의 크기: 25


In [126]:
print(tokenizer.word_index)

{'어터슨': 1, '변호사는': 2, '결코': 3, '환하게': 4, '웃는': 5, '법이': 6, '없는': 7, '묵뚝뚝한': 8, '표정을': 9, '가진': 10, '사람이었다그는': 11, '야윈': 12, '데다가': 13, '키가': 14, '크고': 15, '무미건조하고': 16, '음울해': 17, '보이기까지': 18, '했다그럼에도': 19, '어터슨에게는': 20, '어딘가': 21, '매력적인': 22, '부분이': 23, '있었다': 24}


In [127]:
sequences = list()
for line in text.split('\n'):
  encoded = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(encoded)):
    sequence = encoded[:i+1]
    sequences.append(sequence)

print('학습에 사용할 샘플의 개수: {}'.format(len(sequences)))

학습에 사용할 샘플의 개수: 23


In [128]:
print(sequences)

[[1, 2], [1, 2, 3], [1, 2, 3, 4], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23], [1, 2, 3, 4, 5, 6, 7, 8, 

In [129]:
max_len = max(len(l) for l in sequences)  # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print('샘플의 최대 길이 {}'.format(max_len))

샘플의 최대 길이 24


In [130]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

In [131]:
print(sequences)

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  2]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  2  3]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  2  3  4]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  2  3  4  5]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  2  3  4  5  6]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  2  3  4  5  6  7]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  2  3  4  5  6  7  8]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  2  3  4  5  6  7  8  9]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  2  3  4  5  6  7  8  9 10]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  1  2  3  4  5  6  7  8  9 10 11]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  1  2  3  4  5  6  7  8  9 10 11 12]
 [ 0  0  0  0  0  0  0  0  0  0  0  1  2  3  4  5  6  7  8  9 10 11 12 13]
 [ 0  0  0  0  0  0  0  0  0  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
 [ 0  0  0  0  0  0  0  0

In [132]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [133]:
print(X)

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  2]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  2  3]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  2  3  4]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  2  3  4  5]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  2  3  4  5  6]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  2  3  4  5  6  7]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  2  3  4  5  6  7  8]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  2  3  4  5  6  7  8  9]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  1  2  3  4  5  6  7  8  9 10]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  1  2  3  4  5  6  7  8  9 10 11]
 [ 0  0  0  0  0  0  0  0  0  0  0  1  2  3  4  5  6  7  8  9 10 11 12]
 [ 0  0  0  0  0  0  0  0  0  0  1  2  3  4  5  6  7  8  9 10 11 12 13]
 [ 0  0  0  0  0  0  0  0  0  1  2  3  4  5  6  7  8  9 10 11 12

In [134]:
print(y)

[ 2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]


In [135]:
y = to_categorical(y, num_classes=vocab_size)

In [136]:
print(y)

[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [137]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [138]:
embedding_dim = 10
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 1s - loss: 3.2271 - accuracy: 0.0435 - 1s/epoch - 1s/step
Epoch 2/200
1/1 - 0s - loss: 3.2088 - accuracy: 0.0870 - 14ms/epoch - 14ms/step
Epoch 3/200
1/1 - 0s - loss: 3.1917 - accuracy: 0.0870 - 11ms/epoch - 11ms/step
Epoch 4/200
1/1 - 0s - loss: 3.1753 - accuracy: 0.0870 - 13ms/epoch - 13ms/step
Epoch 5/200
1/1 - 0s - loss: 3.1593 - accuracy: 0.0870 - 12ms/epoch - 12ms/step
Epoch 6/200
1/1 - 0s - loss: 3.1432 - accuracy: 0.1739 - 11ms/epoch - 11ms/step
Epoch 7/200
1/1 - 0s - loss: 3.1267 - accuracy: 0.2174 - 17ms/epoch - 17ms/step
Epoch 8/200
1/1 - 0s - loss: 3.1096 - accuracy: 0.2174 - 10ms/epoch - 10ms/step
Epoch 9/200
1/1 - 0s - loss: 3.0918 - accuracy: 0.1304 - 11ms/epoch - 11ms/step
Epoch 10/200
1/1 - 0s - loss: 3.0731 - accuracy: 0.1304 - 10ms/epoch - 10ms/step
Epoch 11/200
1/1 - 0s - loss: 3.0534 - accuracy: 0.1304 - 11ms/epoch - 11ms/step
Epoch 12/200
1/1 - 0s - loss: 3.0326 - accuracy: 0.1304 - 13ms/epoch - 13ms/step
Epoch 13/200
1/1 - 0s - loss: 3.0108 - ac

In [139]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        # 현재 단어에 대한 정수 인코딩과 패딩
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
        # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면 break
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [141]:
print(sentence_generation(model, tokenizer, '묵뚝뚝한', 3))

묵뚝뚝한 야윈 야윈 데다가


In [144]:
print(sentence_generation(model, tokenizer, '음울해', 2))

음울해 야윈 했다그럼에도


In [146]:
print(sentence_generation(model, tokenizer, '결코', 5))

결코 야윈 웃는 법이 없는 어터슨에게는
